In [2]:
import numpy as np
from tqdm import tqdm_notebook

from sdcdup.utils import overlap_tag_pairs
from sdcdup.utils import overlap_tag_maps
from sdcdup.utils import load_duplicate_truth

from sdcdup.features.image_features import SDCImageContainer
from sdcdup.features.image_features import load_image_overlap_properties

%reload_ext autoreload
%autoreload 2

# SENDTOENV
image_shipcnt_grids_file = 'data/interim/image_shipcnt_grids.pkl'

In [6]:
sdcic = SDCImageContainer()
sdcic.preprocess_label_properties(
    image_shipcnt_grids_file)

100%|██████████| 192555/192555 [00:00<00:00, 414534.69it/s]


In [10]:
dup_truth = load_duplicate_truth()
print(len(dup_truth))

128950


In [8]:
# score_types = ['bmh', 'cmh', 'con', 'hom', 'eng', 'cor', 'epy', 'enp', 'pix', 'px0', 'shp']
n_matching_tiles_list = [9, 6, 4, 3, 2, 1]
overlap_image_maps = load_image_overlap_properties(n_matching_tiles_list)
print(len(overlap_image_maps))

100%|██████████| 75936/75936 [00:01<00:00, 49248.88it/s]

227355


## Find overlaps with ships

In [12]:
untested_image_pairs_with_ship_masks = []
for (img1_id, img2_id), overlap_maps in tqdm_notebook(overlap_image_maps.items()):
    # TODO: Find out which remaining tile pairs have masks but aren't in dup_truth.

    mask1 = sdcic.tile_shipcnt_grids[img1_id]
    mask2 = sdcic.tile_shipcnt_grids[img2_id]
    
    has_mask1 = np.sum(mask1) > 0
    has_mask2 = np.sum(mask2) > 0

    if not (has_mask1 and has_mask2):
        continue

    for img1_overlap_tag in overlap_maps:
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            continue
        untested_image_pairs_with_ship_masks.append((img1_id, img2_id))
        break

len(overlap_image_maps), len(untested_image_pairs_with_ship_masks)

(227355, 96065)

In [11]:
untested_overlaps_with_ship_masks = []
for (img1_id, img2_id), overlap_maps in tqdm_notebook(overlap_image_maps.items()):
    # TODO: Find out which remaining tile pairs have masks but aren't in dup_truth.

    mask1 = sdcic.tile_shipcnt_grids[img1_id]
    mask2 = sdcic.tile_shipcnt_grids[img2_id]
    
    has_mask1 = np.sum(mask1) > 0
    has_mask2 = np.sum(mask2) > 0

    if not (has_mask1 and has_mask2):
        continue

    for img1_overlap_tag in overlap_maps:
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            continue

        mask1_slice_total = np.sum(mask1[overlap_tag_maps[img1_overlap_tag]])
        mask2_slice_total = np.sum(mask2[overlap_tag_maps[overlap_tag_pairs[img1_overlap_tag]]])

        if mask1_slice_total + mask2_slice_total < 1:
            continue

        untested_overlaps_with_ship_masks.append((img1_id, img2_id, img1_overlap_tag))

len(overlap_image_maps), len(untested_overlaps_with_ship_masks)

(227355, 85589)